In [17]:
import import_ipynb
import copy
from builtins import range
import cubiecube as QBC
import corner as Co
import edge as Ed
import facelet as Fc
import facecube as FC
import numpy as np

In [18]:
def count_RB(not_picked, candi):
    count = 0
    for i in range(len(not_picked)):
        if not not_picked[i] and candi < i:
            count += 1
    return count 
    
    
def Corner_coordinateToPermutation(a = [0,0,0,0,0,0,0]):
    b = a[::-1]
    ans = [0 for i in range(8)]
    ans[0] = 7 - b[0]
    not_picked = [True  for i in range(8)]
    not_picked[ans[0]] = False
    for i in range(1, len(ans)-1):
        for candidate in range(7-b[i]+1):
            RB = count_RB(not_picked, candidate)
            LB = b[i]
            #print("RB=", RB, " LB=", LB)
            if not_picked[candidate] and LB + RB + candidate == 7:
                ans[i] = candidate
                not_picked[ans[i]] = False
    for i in range(len(not_picked)):
        if not_picked[i]:
            ans[-1] = i

    return ans[::-1]

def Edge_coordinateToPermutation(a = [0,0,0,0,0,0,0,0,0,0,0]):
    a = list(a)
    b = a[::-1]
    ans = [0 for i in range(12)]
    ans[0] = 11 - b[0]
    not_picked = [True  for i in range(12)]
    not_picked[ans[0]] = False
    for i in range(1, len(ans)-1):
        for candidate in range(11-b[i]+1):
            RB = count_RB(not_picked, candidate)
            LB = b[i]
            # RB : 右邊比他大的數量  LB : 左邊比他大的數量
            if not_picked[candidate] and LB + RB + candidate == 11:
                ans[i] = candidate
                not_picked[ans[i]] = False
    for i in range(len(not_picked)):
        if not_picked[i]:
            ans[-1] = i
    return ans[::-1]

class CoordinateCube():
    def __init__(self, cp = 0, co =0, ep = 0, eo = 0, **kwarg):
        # we ignore the DRB entry since it can be determined by other terms.
        """
        We use numbers to record the orientation and permutation.
        
        By expressing co and eo in trinary and binary base respectively, we'll get the first 7 and 11 orientations.
        The other orientation can be determined by whether it's divided by 3 or 2
        
        By expressing cp and ep in variable base, we'll get the permutation except the first one
        The first can be easily determined by checking the other element.
        
        0 <= co < 3^7
        0 <= eo < 2^11
        0 <= cp < 8!
        0 <= ep < 12!
        
        """
        self.cp = cp
        self.co = co
        self.ep = ep
        self.eo = eo
        if kwarg['CubieCube'] is not None:
            self.cp, self.co, self.ep, self.eo = self.QB_to_Coor(kwarg['CubieCube'])
    
    def QB_to_Coor(self, QB):
        """
        We assign four numbers for the orientation and permutations.
        
        For corner orientation, we ignore DRB and express the orientation in trinary base.
        For edge orientation, we ignore BR and express the orientation in binary base.
        For the permutation, we ignore URF, UR and express them in variable base.
        
        Here we explain how we assign numbers to each digit.
        First we define a (partial) order of corner and edge cubies:
        URF　<　UFL　<　ULB　<　UBR　<　DFR　<　DLF　<　DBL　<　DRB and 
        UR < UF < UL < UB < DR < DF < DL < DB < FR < FL < BL <　BR
        Note that these are orders defined in our corner and edge notations.
        We don't assign numbers to URF and UR
        For other places, we count the number of all corners(edge) left of XXX (XX), whose orders are higher than the order of XXX(XX).
        Then we express them in variable basis, that is, c_1*1! + c_2*2! + c_3*3! + ... + c_n*n!
        
        These four numbers, cp, co, ep, eo, lie in the following range:
        0 <= co < 3^7
        0 <= eo < 2^11
        0 <= cp < 8!
        0 <= ep < 12!
        
        """
        
        cp_coor = 0
        co_coor = 0
        ep_coor = 0
        eo_coor = 0
        # Corner
        for i in range(len(QB.co) - 1):
            co_coor += QB.co[i] * (3**(7-i-1))
        
        for i in range(1, len(QB.cp)):
            # count the the number of all corners left of XXX, whose orders are higher than the order of XXX.
            count = 0
            for j in range(i): 
                if QB.cp[j] > QB.cp[i]:
                    count += 1
            cp_coor += count * np.math.factorial(i)
        
        # Edge
        for i in range(len(QB.eo) - 1):
            eo_coor += QB.eo[i] * (2**(11-i-1))
        
        for i in range(1, len(QB.ep)):
            # count the the number of all edges left of XX, whose orders are higher than the order of XX.
            count = 0
            for j in range(i): 
                if QB.ep[j] > QB.ep[i]:
                    count += 1
            ep_coor += count * np.math.factorial(i)
        
        return [cp_coor ,co_coor, ep_coor, eo_coor] 
        
    
    def __str__(self):
        return str(self.to_cubiecube())
    
    def to_2dstring(self):
        return self.to_cubiecube().to_2dstring()
    
     
    def to_cubiecube(self):
        
        cp = []
        pre_cp = []               # we need to interpret the digits in variable base into permutation
        n_cp = self.cp
        for i in range(1,8):        # assign the permutations except URF
            pre_cp.append(n_cp%np.math.factorial(i+1))
        pre_cp = np.array(pre_cp)
        pre_cp[1:] =  pre_cp[1:8] - pre_cp[:-1]
        pre_cp = pre_cp / np.array([ np.math.factorial(i) for i in range(1, 8) ])
        cp = Corner_coordinateToPermutation(pre_cp.astype(int))
        
        
        ep = []
        pre_ep = []               # we need to interpret the digits in variable base into permutation
        n_ep = self.ep
        for i in range(1,12):        # assign the permutations except URF
            pre_ep.append(n_ep%np.math.factorial(i+1))
        pre_ep = np.array(pre_ep)
        pre_ep[1:] =  pre_ep[1:12] - pre_ep[:-1]
        pre_ep = pre_ep / np.array([ np.math.factorial(i) for i in range(1, 12) ])
        ep = Edge_coordinateToPermutation(pre_ep.astype(int))

        co = [0 for i in range(8)]
        n_co = self.co
        count_co = 0
        for i in range(7):        # assign first 7 orientations
            count_co += n_co%3
            co[6 - i] = n_co%3
            n_co //= 3
        co[7] = (-count_co)%3 # sum of all entries must be divided by 3
        
        eo = [0 for i in range(11)]
        n_eo = self.eo
        count_eo = 0
        for i in range(11):       # assign first 11 orientations
            count_eo += n_eo%2
            eo[10-i] = n_eo%2
            n_eo //= 2
        eo.append(count_eo%2)     # sum of all entries must be divided by 2   
        
        Q = QBC.CubieCube(cp, np.array(co), ep, np.array(eo))
        return Q

In [19]:
cube_in_CoordinateCube_level = CoordinateCube(CubieCube = None)
print(f"cube in CoordinateCube level:\n\
ep = {cube_in_coordinate_level.ep}\n\
eo = {cube_in_coordinate_level.eo}\n\
cp = {cube_in_coordinate_level.co}\n\
co = {cube_in_coordinate_level.co}\n")

cubiecube = cube_in_CoordinateCube_level.to_cubiecube()
print(f"convert CoordinateCube level to CubieCube level: \n\
cp = {cubiecube.cp}\n\
co = {cubiecube.co}\n\
ep = {cubiecube.ep}\n\
eo = {cubiecube.eo}\n" )

cube in CoordinateCube level:
ep = 0
eo = 0
cp = 0
co = 0

convert CoordinateCube level to CubieCube level: 
cp = [0, 1, 2, 3, 4, 5, 6, 7]
co = [0 0 0 0 0 0 0 0]
ep = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
eo = [0 0 0 0 0 0 0 0 0 0 0 0]



In [23]:
CubieCube_level = QBC.CubieCube()
print(f"cube in CubieCube level: \n\
cp = {cubiecube.cp}\n\
co = {cubiecube.co}\n\
ep = {cubiecube.ep}\n\
eo = {cubiecube.eo}\n" )

CoordCube = CoordinateCube(CubieCube = cube_in_CubieCube_level)
print(f"convert CubieCube level to CoordinateCube level:\n\
ep = {CoordCube.ep}\n\
eo = {CoordCube.eo}\n\
cp = {CoordCube.co}\n\
co = {CoordCube.co}\n")

cube in CubieCube level: 
cp = [0, 1, 2, 3, 4, 5, 6, 7]
co = [0 0 0 0 0 0 0 0]
ep = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
eo = [0 0 0 0 0 0 0 0 0 0 0 0]

convert CubieCube level to CoordinateCube level:
ep = 0
eo = 0
cp = 0
co = 0



In [42]:
# way to move
Q = CoordinateCube(0, 0, 0, 0, CubieCube = None)
Q_C = Q.to_cubiecube()
print(f"Q is in CoordinateCube level: \n\
cp = {Q.cp}  co = {Q.co}\n\
ep = {Q.ep}  eo = {Q.eo}\n" )

Q_C.move("R")
print(f"cube in CubieCube level after R move: \n\
cp = {Q_C.cp}\nco = {Q_C.co}\n\
ep = {Q_C.ep}\neo = {Q_C.eo}\n" )

Q_2 = CoordinateCube(CubieCube = Q_C)
print(f"Q_2 is Q after R move: \n\
cp = {Q_2.cp}  co = {Q_2.co}\n\
ep = {Q_2.ep}  eo = {Q_2.eo}\n" )
print(f"cube-like demonstration: \n{Q_2.to_2dstring()}")

Q is in CoordinateCube level: 
cp = 0  co = 0
ep = 0  eo = 0

cube in CubieCube level after R move: 
cp = [4, 1, 2, 0, 7, 5, 6, 3]
co = [2 0 0 1 1 0 0 2]
ep = [8, 1, 2, 3, 11, 5, 6, 7, 4, 9, 10, 0]
eo = [0 0 0 0 0 0 0 0 0 0 0 0]

Q_2 is Q after R move: 
cp = 21021  co = 1494
ep = 443289849  eo = 0

cube-like demonstration: 
    |UUF|
    |UUF|
    |UUF|
|LLL|FFD|RRR|UBB|
|LLL|FFD|RRR|UBB|
|LLL|FFD|RRR|UBB|
    |DDB|
    |DDB|
    |DDB|



In [43]:
Q = CoordinateCube(0, 0, 0, 0, CubieCube = None)
Q_C = Q.to_cubiecube()
Q_C.move("RUUDFBBLULFRDD")
Q_2 = CoordinateCube(CubieCube = Q_C)
print(f"Q_2 is Q after RUUDFBBLULFRDD move: \n\
cp = {Q_2.cp}\nco = {Q_2.co}\n\
ep = {Q_2.ep}\neo = {Q_2.eo}\n" )
print(f"{Q_2.to_2dstring()}")

Q_2 is Q after RUUDFBBLULFRDD move: 
cp = 27865
co = 156
ep = 16511564
eo = 1025

    |FUR|
    |DUL|
    |UFU|
|DBF|RLR|BUD|BRL|
|DLR|DFD|FRR|BBL|
|RLU|FFL|DRL|BBF|
    |LUB|
    |BDF|
    |DUU|

